In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pathlib import Path
import math
from matplotlib.colors import ListedColormap

from datetime import time, timedelta, datetime, date


# Definição de diretórios e Disco de leitura dos arquivos (Desktop ou Notebook) 
config_dirs = open("prefixo_dados.txt").readlines() 
DIR_PREFIXO = Path(config_dirs[0].strip())


DIRETORIO_PRINCIPAL =  DIR_PREFIXO / "Projeto_PA" / "Projeto_PA_validado"
DIRETORIO_DATASET = DIR_PREFIXO / "Projeto_PA"

DIR_ABORDAGENS = DIRETORIO_PRINCIPAL / "analise_abordagens"
DIR_KMEANS = DIRETORIO_PRINCIPAL / "KMeansRandom"
DIR_NMF = DIRETORIO_PRINCIPAL / "NMF"



In [2]:
caminho = DIRETORIO_PRINCIPAL / DIR_KMEANS
caminho
diretorios = {'kmeans':DIR_KMEANS, 'nmf':DIR_NMF}

nome_abordagem = {'cadastro':"_approach_cadastro", 'doenca':"_approach_doenca",
                  'medidas':"_approach_medidas",'estatistica':"_approach_estatistica",
                 'escolho_doenca':"_approach_escolho_doenca", 'categorias':"_approach_classes",
                 'medidas_masc':"_approach_medidas_masc",'medidas_fem':"_approach_medidas_fem"}

In [3]:
#DIR_DICT / f"Dict_approach_doenca
#_grupo_{grupo}.csv", 

# Análise é uma opção do dicionário criado na célula acima
algoritmo = input("Informe a analise ")

caminho_medidas = diretorios[algoritmo.lower()]/ f"{algoritmo.upper()}{nome_abordagem['medidas']}"
caminho_estatistica = diretorios[algoritmo.lower()]/ f"{algoritmo.upper()}{nome_abordagem['estatistica']}"
caminho_doenca = diretorios[algoritmo.lower()]/ f"{algoritmo.upper()}{nome_abordagem['doenca']}"
caminho_cadastro = diretorios[algoritmo.lower()]/ f"{algoritmo.upper()}{nome_abordagem['cadastro']}"
caminho_escolho_doenca = diretorios[algoritmo.lower()]/ f"{algoritmo.upper()}{nome_abordagem['escolho_doenca']}"
caminho_categorias = diretorios[algoritmo.lower()]/ f"{algoritmo.upper()}{nome_abordagem['categorias']}"
caminho_medidas_masc = diretorios[algoritmo.lower()]/ f"{algoritmo.upper()}{nome_abordagem['medidas_masc']}"
caminho_medidas_fem = diretorios[algoritmo.lower()]/ f"{algoritmo.upper()}{nome_abordagem['medidas_fem']}"


Informe a analise kmeans


In [4]:
ls_abordagens = list()
ls_nomes = list()

try:
    ls_abordagens.append(pd.read_csv(f"{caminho_estatistica}_todos_pacientes.csv", sep=";", encoding='latin1'))
    ls_nomes.append('estatistica')
except:
    pass

try:
    ls_abordagens.append(pd.read_csv(f"{caminho_medidas}_todos_pacientes.csv", sep=";", encoding='latin1'))
    ls_nomes.append('medidas')
except:
    pass

try:
    ls_abordagens.append(pd.read_csv(f"{caminho_doenca}_todos_pacientes.csv", sep=";", encoding='latin1'))
    ls_nomes.append('doenca')
except:
    pass

try:
    ls_abordagens.append(pd.read_csv(f"{caminho_cadastro}_todos_pacientes.csv", sep=";", encoding='latin1'))
    ls_nomes.append('cadastro')
except:
    pass

try:
    ls_abordagens.append(pd.read_csv(f"{caminho_escolho_doenca}_todos_pacientes.csv", sep=";", encoding='latin1'))
    ls_nomes.append('escolho_doenca')
except:
    pass

try:
    ls_abordagens.append(pd.read_csv(f"{caminho_categorias}_todos_pacientes.csv", sep=";", encoding='latin1'))
    ls_nomes.append('categorias')
except:
    pass

try:
    ls_abordagens.append(pd.read_csv(f"{caminho_medidas_masc}_todos_pacientes.csv", sep=";", encoding='latin1'))
    ls_nomes.append('medidas_masc')
except:
    pass

try:
    ls_abordagens.append(pd.read_csv(f"{caminho_medidas_fem}_todos_pacientes.csv", sep=";", encoding='latin1'))
    ls_nomes.append('medidas_fem')
except:
    pass

In [5]:
#print(f"{caminho_doenca}_grupo_{1}.csv")


## Funções

In [6]:
def faixas_imc(imc):
    classes = ["baixo peso","normal","sobrepeso",
               "obesidade grau I","obesidade grau II","obesidade Grau III"]
    intervalos = [(0, 18),(18, 25),(25, 30),(30, 35),(35, 40),(40, 1000)]
    
    for faixa in range(len(classes)):
        if imc < intervalos[faixa][1]:
            return classes[faixa]
        
def faixas_imc_classe(imc):
    classes = [1, 2, 3, 4, 5, 6]
    intervalos = [(0, 18),(18, 25),(25, 30),(30, 35),(35, 40),(40, 1000)]
    
    for faixa in range(len(classes)):
        if imc < intervalos[faixa][1]:
            return classes[faixa]
    
def faixas_etarias(idade):
    classes = ["Jovens","Adultos","Meia-idade","Idoso", "Ancião","Velhice extrema"]
    intervalos = [(0, 20),(20, 45),(45, 60),(60, 75),(75, 90),(90, 200)]
    
    for faixa in range(len(classes)):
        if idade <= intervalos[faixa][1]:
            return classes[faixa]

def faixas_etarias_classe(idade):
    classes = [1, 2, 3, 4, 5, 6]
    intervalos = [(0, 20),(20, 45),(45, 60),(60, 75),(75, 90),(90, 200)]
    
    for faixa in range(len(classes)):
        if idade <= intervalos[faixa][1]:
            return classes[faixa]

def faixa_genero(genero):
    if genero == 'M':
        return 0
    else:
        return 1

def verifica_grupo(coluna,grupo):
    saida = list()
    
    for linha in coluna:
        if linha == grupo:
            saida.append(1)
        else:
            saida.append(0)
    return saida

genero_1_0 = {'genero':{'M':0,'F':1}}

def modifica_df(df,abordagem):
    df = df.loc[:,:'diast 8:45']
    grupos  = set(df['grupo'])
    for grupo in grupos:
        df[f'cluster{grupo}'] = verifica_grupo(df['grupo'],grupo)
    df = df.replace(genero_1_0)
    df['IMC'] = df['IMC'].apply(faixas_imc_classe)
    df['idade'] = df['idade'].apply(faixas_etarias_classe)
    df = df.drop(['data_nascimento','peso','altura','grupo','data_exame'],axis=1)
    
    df.to_csv(DIR_ABORDAGENS / "todos_pacientes" / f'todos_pacientes_{abordagem}.csv', 
                           sep=";", encoding="latin1", index=False)
    return df


In [7]:
ls_abordagens[6]

,numero_identificacao,data_nascimento,genero,idade,peso,altura,IMC,data_exame,grupo,sist 9:00,...,Mediana Diastolica em 24h,AUC Sistolica em 24h,AUC Diastolica em 24h,AUC PP Daytime,AUC PP Nighttime,DP Sistolica em 24h,DP Diastolica em 24h,Diferenca % Dipping,Picos Sistolica,Picos Diastolica
0,4,1923-10-05,M,94.0,85.0,160,33.2,2018-09-10 12:25:23,6,105.0,...,59.0,9545.5,4902.0,3748.0,841.5,16.52,11.36,0.966,11,34
1,5,1948-10-07,M,70.0,80.0,165,29.4,2018-12-26 07:39:41,6,103.0,...,62.0,8586.5,5034.5,2915.0,606.0,7.51,6.65,0.969,9,23
2,6,1999-03-23,M,19.0,100.0,197,25.8,2018-12-26 08:06:19,1,147.0,...,79.0,10674.0,6154.0,3721.0,751.5,12.70,13.01,1.056,19,19
3,8,1990-03-07,M,28.0,120.0,172,40.6,2018-12-26 08:26:59,4,113.0,...,78.0,9141.5,6118.0,2405.0,566.0,12.27,12.79,1.137,10,11
4,9,1982-04-28,M,36.0,100.0,185,29.2,2018-12-26 09:56:35,3,152.0,...,104.5,11583.0,8198.0,2783.5,558.5,10.83,12.82,1.051,37,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,21775,1955-09-04,M,65.0,94.0,"1,83",28.1,2020-09-10 14:40:03,4,105.0,...,78.0,9742.5,6171.5,2984.5,546.0,14.42,8.85,1.020,13,6
8756,21783,1969-05-24,M,51.0,93.0,168,33.0,2020-09-10 15:40:30,4,119.0,...,74.0,9121.5,5963.5,2605.0,513.0,14.69,11.86,1.214,17,16
8757,21786,1972-11-25,M,47.0,75.0,165,27.5,2020-09-10 16:03:08,3,163.0,...,116.0,12224.5,9265.0,2439.0,474.5,18.66,14.80,1.134,49,60
8758,21787,1970-10-19,M,49.0,90.0,183,26.9,2020-09-10 16:21:02,5,114.0,...,85.0,10970.0,6819.0,3319.0,738.0,17.22,13.67,1.046,32,27


In [8]:
for abordagem, nome in zip(ls_abordagens,ls_nomes):
    modifica_df(abordagem,nome)

De todos os pacientes tratados, gravar csv de todos que tem um determinado padrão (ex: Dipping =True), resultando em cluster_todos_dipping, cluster_todos_non_dipping, cluster_todos_whitecoat,.......
Calcular a Associação de cluster_todos_xxxxx com C1 a C7